In [257]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score
import sklearn 
sklearn.__version__

'1.1.3'

In [153]:
import os
os.getcwd()

'E:\\test'

In [156]:
bd_train=pd.read_csv(r"E:\test\Consumer_Complaints_train.csv")
bd_test=pd.read_csv(r"E:\test\Consumer_Complaints_test.csv") 

In [158]:
 #bd_train.describe()

In [159]:
bd_test['Consumer disputed?']=np.nan

In [160]:
bd_train['data'] = 'train'
bd_test['data'] = 'test'
bd_test=bd_test[bd_train.columns]
bd_all=pd.concat([bd_train,bd_test],axis=0)

In [161]:
bd_all.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103,train
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666,train
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363,train
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177,train
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572,train


In [162]:
list(zip(bd_all.columns,bd_all.dtypes,bd_all.nunique()))

[('Date received', dtype('O'), 1759),
 ('Product', dtype('O'), 12),
 ('Sub-product', dtype('O'), 47),
 ('Issue', dtype('O'), 95),
 ('Sub-issue', dtype('O'), 68),
 ('Consumer complaint narrative', dtype('O'), 92118),
 ('Company public response', dtype('O'), 10),
 ('Company', dtype('O'), 3437),
 ('State', dtype('O'), 62),
 ('ZIP code', dtype('O'), 27342),
 ('Tags', dtype('O'), 3),
 ('Consumer consent provided?', dtype('O'), 4),
 ('Submitted via', dtype('O'), 6),
 ('Date sent to company', dtype('O'), 1708),
 ('Company response to consumer', dtype('O'), 7),
 ('Timely response?', dtype('O'), 2),
 ('Consumer disputed?', dtype('O'), 2),
 ('Complaint ID', dtype('int64'), 598027),
 ('data', dtype('O'), 2)]

In [163]:
bd_all['Consumer disputed?'].value_counts()

No     376990
Yes    101431
Name: Consumer disputed?, dtype: int64

In [164]:
bd_all['Consumer disputed?']=np.where(bd_all['Consumer disputed?']=='Yes',1,0)
#bd_all['Consumer disputed?']=np.where(bd_all['Consumer disputed?'][:1]=='no',0,bd_all['Consumer disputed?'])
#bd_all['Consumer disputed?']=pd.to_numeric(bd_all['Consumer disputed?'],errors='coerce')

In [165]:
bd_all['Consumer disputed?'].value_counts()

0    496596
1    101431
Name: Consumer disputed?, dtype: int64

In [166]:
bd_all.columns=[re.sub(r'\W+','_',i).lower() for i in bd_all.columns]

In [167]:
bd_all.columns

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'consumer_complaint_narrative', 'company_public_response', 'company',
       'state', 'zip_code', 'tags', 'consumer_consent_provided_',
       'submitted_via', 'date_sent_to_company', 'company_response_to_consumer',
       'timely_response_', 'consumer_disputed_', 'complaint_id', 'data'],
      dtype='object')

In [168]:
bd_all["date_received"]=pd.to_datetime(bd_all.date_received,format="%Y-%m-%d")

In [169]:
bd_all["date_sent_to_company"]=pd.to_datetime(bd_all.date_sent_to_company,format="%Y-%m-%d")

In [170]:
bd_all["dif_days"]=(bd_all["date_sent_to_company"]-bd_all["date_received"]).dt.days

In [171]:
bd_all["dif_days"]

0          1
1          6
2         21
3          0
4          8
          ..
119601     6
119602     0
119603     3
119604     1
119605     0
Name: dif_days, Length: 598027, dtype: int64

In [172]:
bd_all.tags.isnull().sum()

513950

In [173]:
bd_all.zip_code.str.slice(0,1)

0         4
1         1
2         9
3         3
4         0
         ..
119601    6
119602    9
119603    9
119604    1
119605    9
Name: zip_code, Length: 598027, dtype: object

In [174]:
bd_all["state"]=bd_all.zip_code.str.slice(0,1)

In [175]:
bd_all["area"]=bd_all.zip_code.str.slice(0,2)

In [176]:
bd_all.drop(['company'], axis=1, inplace=True)

In [177]:
bd_all.consumer_complaint_narrative.isnull().sum()

504376

In [178]:
bd_all.consumer_consent_provided_.isnull().sum()

428676

In [179]:
bd_all.columns

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'consumer_complaint_narrative', 'company_public_response', 'state',
       'zip_code', 'tags', 'consumer_consent_provided_', 'submitted_via',
       'date_sent_to_company', 'company_response_to_consumer',
       'timely_response_', 'consumer_disputed_', 'complaint_id', 'data',
       'dif_days', 'area'],
      dtype='object')

In [180]:
bd_all.timely_response_.value_counts()

Yes    587954
No      10073
Name: timely_response_, dtype: int64

In [181]:
drop_calls=["date_received","date_sent_to_company","complaint_id","tags","zip_code","consumer_complaint_narrative"]

In [182]:
bd_all.drop(drop_calls,axis=1,inplace=True)

In [183]:
bd_all.columns

Index(['product', 'sub_product', 'issue', 'sub_issue',
       'company_public_response', 'state', 'consumer_consent_provided_',
       'submitted_via', 'company_response_to_consumer', 'timely_response_',
       'consumer_disputed_', 'data', 'dif_days', 'area'],
      dtype='object')

In [184]:
bd_all.columns

Index(['product', 'sub_product', 'issue', 'sub_issue',
       'company_public_response', 'state', 'consumer_consent_provided_',
       'submitted_via', 'company_response_to_consumer', 'timely_response_',
       'consumer_disputed_', 'data', 'dif_days', 'area'],
      dtype='object')

In [185]:
si=SimpleImputer(strategy="constant")

In [186]:
si_transform=si.fit_transform(bd_all.select_dtypes(object).drop('data',axis=1))
##  This is where you need to change it you haven't done it !!!!

In [187]:
si_transform=pd.DataFrame(si_transform)

In [188]:
bd_all.select_dtypes(object).drop('data',axis=1).columns

Index(['product', 'sub_product', 'issue', 'sub_issue',
       'company_public_response', 'state', 'consumer_consent_provided_',
       'submitted_via', 'company_response_to_consumer', 'timely_response_',
       'area'],
      dtype='object')

In [189]:
si_transform.columns=bd_all.select_dtypes(object).drop('data',axis=1).columns

In [190]:
si_transform.head()

,product,sub_product,issue,sub_issue,company_public_response,state,consumer_consent_provided_,submitted_via,company_response_to_consumer,timely_response_,area
0,Credit card,missing_value,Billing statement,missing_value,missing_value,4,missing_value,Web,Closed with explanation,Yes,48
1,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",missing_value,missing_value,1,missing_value,Referral,Closed,Yes,18
2,Credit reporting,missing_value,Incorrect information on credit report,Account status,missing_value,9,missing_value,Referral,Closed with non-monetary relief,Yes,92
3,Credit card,missing_value,Billing statement,missing_value,Company chooses not to provide a public response,3,Consent provided,Web,Closed with monetary relief,Yes,30
4,Credit card,missing_value,Transaction issue,missing_value,missing_value,0,missing_value,Web,Closed with explanation,Yes,02


In [197]:
ohe=OneHotEncoder(handle_unknown="ignore",drop="first")


In [198]:
ohe_transform=ohe.fit_transform(si_transform)


In [199]:
ohe.fit_transform(si_transform)


<598027x395 sparse matrix of type '<class 'numpy.float64'>'
	with 6376845 stored elements in Compressed Sparse Row format>

In [200]:
ohe_table=pd.DataFrame.sparse.from_spmatrix(ohe_transform)


In [201]:
ohe_table.columns = ohe.get_feature_names_out()

In [202]:
final_data=pd.concat([ohe_table.reset_index(drop=True),
                      bd_all.select_dtypes(np.number).reset_index(drop=True),
                      bd_all[["data"]].reset_index(drop=True)], axis=1)

In [203]:
final_data.head()

,product_Consumer Loan,product_Credit card,product_Credit reporting,product_Debt collection,product_Money transfers,product_Mortgage,product_Other financial service,product_Payday loan,product_Prepaid card,product_Student loan,...,area_[3,area_[4,area_[7,area_[9,area_`2,area_`3,area_missing_value,consumer_disputed_,dif_days,data
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,train
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,6,train
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,21,train
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,train
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,8,train


In [204]:
final_data.tail()

,product_Consumer Loan,product_Credit card,product_Credit reporting,product_Debt collection,product_Money transfers,product_Mortgage,product_Other financial service,product_Payday loan,product_Prepaid card,product_Student loan,...,area_[3,area_[4,area_[7,area_[9,area_`2,area_`3,area_missing_value,consumer_disputed_,dif_days,data
598022,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,6,test
598023,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,test
598024,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,test
598025,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,test
598026,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,test


In [205]:
train=final_data.query('data=="train"')

In [206]:
train.isnull().sum().sum()

0

In [207]:
test=final_data.query('data=="test"')

In [209]:
train1 = train.drop(["consumer_disputed_",'data'],axis=1)

In [210]:
train1.shape

(478421, 396)

In [211]:
X_train,X_test,y_train,y_test=train_test_split(train1,train.consumer_disputed_.values,
                                             test_size=.20,random_state=1)

In [212]:
X_train.shape

(382736, 396)

In [213]:
X_test.shape

(95685, 396)

In [214]:
X_train.isnull().sum().sum()

0

In [215]:
params={'class_weight':['balanced',None],
 'penalty':['l1'],
 'C':np.linspace(0.01,1000,10)}

In [216]:
model = LogisticRegression(solver='liblinear', penalty='l1')

In [218]:
model.fit(X_train, y_train)

C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


LogisticRegression(penalty='l1', solver='liblinear')

In [228]:
model = RandomForestClassifier()
param_grid = {
    'n_estimators': [30, 50, 70, 100, 150, 200],
    'max_depth': [2, 5, 8, 10],
    'min_samples_split':(.1, .2 , .3, 5, .6, .7),
    'bootstrap': [True],
    'max_features':['sqrt']
}

In [229]:
X_train.shape

(382736, 396)

In [230]:
X_test.shape

(95685, 396)

In [232]:
random_search = RandomizedSearchCV(
  model,
  param_grid, 
  cv=5, 
  n_iter=10, 
  random_state=42)
##random_search.fit(X, y)
##random_search.fit(X_train, y_train)

In [233]:
model=random_search.fit(X_train, y_train)

C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarn

In [234]:
model

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [2, 5, 8, 10],
                                        'max_features': ['sqrt'],
                                        'min_samples_split': (0.1, 0.2, 0.3, 5,
                                                              0.6, 0.7),
                                        'n_estimators': [30, 50, 70, 100, 150,
                                                         200]},
                   random_state=42)

In [235]:
X_train.shape

(382736, 396)

In [116]:
#model.feature_names_in_

In [236]:
model.predict_proba(X_train)[:,1][0:10]

C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


array([0.21972957, 0.21757487, 0.23444692, 0.20979125, 0.21912328,
       0.18249428, 0.2276849 , 0.20838144, 0.24736936, 0.22016942])

In [237]:
model.predict_proba(X_test)[:,1][0:10]

C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


array([0.20082737, 0.20464675, 0.22528173, 0.22630727, 0.17218709,
       0.19359239, 0.23461383, 0.23682231, 0.23841546, 0.24903844])

In [ ]:
##import pickle

##with open('random_search.pkl,'wb') as f:
   ## pickle.dump(random_search, f)

In [ ]:
##with open('random_search.pkl', 'rb') as f:
  ##  loaded_random_search = pickle.load(f)

In [239]:
def report(results, n_top=3):
     for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [263]:
#report(random_search.cv_results_,5)
train_score=model.predict_proba(X_train)[:,1]

C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [242]:
test_score=model.predict_proba(X_test)[:,1]

C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [243]:
#test_score

array([0.20082737, 0.20464675, 0.22528173, ..., 0.23141289, 0.1680073 ,
       0.18135277])

In [249]:
test.drop('data',axis=1, inplace=True)

C:\Users\Pradeep.Kumar\AppData\Local\Temp\ipykernel_18912\2449444815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('data',axis=1, inplace=True)


In [253]:
test.drop('consumer_disputed_',axis=1, inplace=True)

C:\Users\Pradeep.Kumar\AppData\Local\Temp\ipykernel_18912\225319918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('consumer_disputed_',axis=1, inplace=True)


In [254]:
set(test.columns) - set(X_train.columns) 

set()

In [255]:
test_score_final=model.predict_proba(test)[:,1]

C:\env310\py311\Lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [256]:
test_score_final

array([0.24032375, 0.22154906, 0.23605571, ..., 0.19367898, 0.19883867,
       0.21732929])

In [258]:
#test_score_final.to_csv("Sakshi_Dhawan_P3.csv",index=False)

In [260]:
cutoffs=np.linspace(0.01,0.99,99)
#cutoffs

,product_Consumer Loan,product_Credit card,product_Credit reporting,product_Debt collection,product_Money transfers,product_Mortgage,product_Other financial service,product_Payday loan,product_Prepaid card,product_Student loan,...,area_9`,area_[2,area_[3,area_[4,area_[7,area_[9,area_`2,area_`3,area_missing_value,dif_days
98983,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
295629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
364872,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
457200,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
69050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371403,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27
312201,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
267336,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
470924,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [264]:
real = y_train

In [267]:
KS_all={}
for cutoff in cutoffs:

     predicted=(train_score>cutoff).astype(int)
        
     TP=((predicted==1) & (real==1)).sum()
     TN=((predicted==0) & (real==0)).sum()
     FP=((predicted==1) & (real==0)).sum()
     FN=((predicted==0) & (real==1)).sum()

     P=TP+FN
     N=TN+FP


     KS=(TP/P)-(FP/N)

     KS_all[cutoff] = KS

In [270]:
maxv = -9999999
for k, v in KS_all.items():
    if v > maxv:
        maxv = v
        cutoff = k

In [272]:
maxv, cutoff

(0.16101925961330066, 0.21000000000000002)

In [274]:
predicted_class = (train_score > cutoff)+0

In [275]:
actual_class = real

In [277]:
df_class = pd.DataFrame({'predicted_class' : predicted_class,
                        'actual_class' : actual_class})

In [279]:
pd.crosstab(df_class.predicted_class, df_class.actual_class)

actual_class,0,1
predicted_class,,
0,153075,28146
1,148462,53053
